In [2]:
import pandas as pd
from IPython.display import display
import os
import shutil
path = os.chdir('C:/Users/jewel.espiritu/OneDrive - AMDATEX LAS PINAS SERVICES INC/Python/py')
sladf = pd.concat(pd.read_excel('./data/WeeklySLA_Data_2022.xlsx', sheet_name=None), ignore_index=True)

In [3]:
nnewdf = sladf[sladf['Week']==23]
nnewdf = nnewdf.rename( columns = {'TTR Result\n(Excluding Pending Time)': 'TTR'}, inplace = False)
nnewdf = nnewdf.rename( columns = {'TTO Result (Overall)': 'TTO'}, inplace = False )

nnewdf.insert(48, "MetricTTR","TTR Overall", False) #Helper Column
nnewdf.insert(49, "MetricTTO","TTO Overall", False) #Helper Column


In [4]:
TTRpivot = pd.pivot_table(nnewdf, index=['MetricTTR'], columns=['TTR',], values=['Ticket No'], aggfunc='count')
TTRpivot


Ticket No           
TTR         Beyond TAT Within TAT
MetricTTR                        
TTR Overall        617       1521

In [5]:
TTOpivot = pd.pivot_table(nnewdf, index=['MetricTTO'], columns=['TTO',], values=['Ticket No'], aggfunc='count')
TTOpivot


Ticket No           
TTO         Beyond TAT Within TAT
MetricTTO                        
TTO Overall        301       1837

In [6]:
TTRpivot.columns = TTRpivot.columns.droplevel(0)
TTRpivot.columns.name = None
TTRdf = TTRpivot.reset_index()

TTOpivot.columns = TTOpivot.columns.droplevel(0)
TTOpivot.columns.name = None
TTOdf = TTOpivot.reset_index()


In [7]:
TTRBeyond = TTRdf["Beyond TAT"].astype(int)
TTRWithin = TTRdf["Within TAT"].astype(int)
TTRsum = TTRBeyond + TTRWithin
TTRdf["Beyond TAT Percent"] = TTRBeyond / TTRsum
TTRdf["Within TAT Percent"] = TTRWithin / TTRsum

TTOBeyond = TTOdf["Beyond TAT"].astype(int)
TTOWithin = TTOdf["Within TAT"].astype(int)
TTOsum = TTOBeyond + TTOWithin
TTOdf["Beyond TAT Percent"] = TTOBeyond / TTRsum
TTOdf["Within TAT Percent"] = TTOWithin / TTRsum

In [8]:
TTRdf = TTRdf.rename( columns = {'MetricTTR': 'Metric'})

TTOdf = TTOdf.rename( columns = {'MetricTTO': 'Metric'})


In [9]:
frames = [TTRdf, TTOdf]
  
SLA_perf = pd.concat(frames)
display(SLA_perf)

,Metric,Beyond TAT,Within TAT,Beyond TAT Percent,Within TAT Percent
0,TTR Overall,617,1521,0.288587,0.711413
0,TTO Overall,301,1837,0.140786,0.859214


In [10]:
FilePath = r'C:\repo\SLA_Perf.xlsx'
# Create a Pandas Excel writer using XlsxWriter as the engine.
writer = pd.ExcelWriter(FilePath, engine='xlsxwriter')

# Convert the dataframe to an XlsxWriter Excel object.
SLA_perf.to_excel(writer, sheet_name='SLA', index = False)
writer.save()
